In [2]:
import os
import cv2
import glob

# Set these paths as needed
DATASET_BASE = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset"
IMAGE_DIRS = [os.path.join(DATASET_BASE, "train/images"), os.path.join(DATASET_BASE, "valid/images")]
LABEL_DIRS = [os.path.join(DATASET_BASE, "train/labels"), os.path.join(DATASET_BASE, "valid/labels")]
OUTPUT_DIR = os.path.join(DATASET_BASE, "visualized")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Number of keypoints (change if needed)
NUM_KEYPOINTS = 17 + 20  # e.g., 17 human + 20 horse, adjust to your setup

def draw_joints_on_image(img, keypoints, color=(0, 0, 255)):
    for x, y, v in keypoints:
        if v > 0:  # Only plot visible/labeled keypoints
            cv2.circle(img, (int(x), int(y)), 4, color, -1)
    return img

for img_dir, label_dir in zip(IMAGE_DIRS, LABEL_DIRS):
    for img_path in glob.glob(os.path.join(img_dir, "*.jpg")):
        base = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, base + ".txt")
        if not os.path.exists(label_path):
            continue

        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        with open(label_path, "r") as f:
            line = f.readline().strip().split()
            # Skip class and bbox (first 5 entries)
            kpt_entries = line[5:]
            keypoints = []
            for i in range(0, len(kpt_entries), 3):
                if i+2 >= len(kpt_entries): break
                x = float(kpt_entries[i]) * w
                y = float(kpt_entries[i+1]) * h
                v = float(kpt_entries[i+2])
                keypoints.append((x, y, v))

        img = draw_joints_on_image(img, keypoints)
        out_path = os.path.join(OUTPUT_DIR, base + "_joints.jpg")
        cv2.imwrite(out_path, img)
        print(f"Saved: {out_path}")

Saved: /fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset/visualized/000015_joints.jpg
Saved: /fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset/visualized/000030_joints.jpg
Saved: /fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset/visualized/000060_joints.jpg
Saved: /fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset/visualized/000075_joints.jpg
Saved: /fs/nexus-scratch/hwahed/dlcDatasetMaker/dataset/visualized/000045_joints.jpg
